In [1]:
import os
import yaml
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd

In [2]:
def flatten(l):
  return [item for sublist in l for item in sublist]

In [3]:
load_dotenv()
# This gets all env variables starting with RIM_
cf = {}
for e in os.environ:
  if e.startswith('RIM_'):
    e_name = e.replace('RIM_', '').lower()
    cf[e_name] = os.getenv(e)
# This tries to convert port numbers to int
cf = {k: (int(v) if v.isnumeric() else v) for k, v in cf.items()}

In [4]:
mc = MongoClient(
  host=cf['mongo_host'],
  port=cf['mongo_port'],
  username=cf['mongo_user'],
  password=cf['mongo_pass'],
)
mc._timeout = 1

In [5]:
with open('../data/item-categories.yaml', 'r') as f:
  categories = flatten(list(yaml.safe_load_all(f)))
categories

[{'_id': 'foodstuff',
  'desc': 'All you can eat or drink',
  'subcats': ['greens', 'meat', 'meals', 'beverage', 'liquors']},
 {'_id': 'greens', 'desc': 'Grows in a garden'},
 {'_id': 'meat', 'desc': 'Says Moooo'},
 {'_id': 'meals', 'desc': 'Is full of ingredients'},
 {'_id': 'beverage', 'desc': 'Is liquid'},
 {'_id': 'liquors', 'desc': 'Makes you dizzy'},
 {'_id': 'weapons',
  'desc': 'Made for killing',
  'subcats': ['melee', 'ranged', 'exotic']},
 {'_id': 'melee',
  'desc': "To look into someone's eyes",
  'subcats': ['pointed', 'blunt', 'edged']},
 {'_id': 'ranged', 'desc': 'For the introverts'},
 {'_id': 'exotic', 'desc': 'Is it a frying pan?'},
 {'_id': 'pointed', 'desc': 'Spears'},
 {'_id': 'blunt', 'desc': 'Crushes limbs'},
 {'_id': 'edged', 'desc': 'Sharp as razor'},
 {'_id': 'money',
  'desc': 'The medium of exchange',
  'subcats': ['credit', 'valuables']},
 {'_id': 'credit', 'desc': 'Cards and chips'},
 {'_id': 'valuables', 'desc': 'Filthy lucre'},
 {'_id': 'wearables',
  'd

In [6]:
df = pd.DataFrame(categories)
parents = []
for _, row in df.iterrows():
  if type(row['subcats']) == list:
    for cat in row['subcats']:
      parents.append({
        '_id': cat,
        'parent': row['_id']
      })
parents = pd.DataFrame(parents)
categories = pd.concat([df.set_index('_id'), parents.set_index('_id')], axis=1, join='outer').reset_index()
categories.fillna('', inplace=True)
categories

,_id,desc,subcats,parent
0,foodstuff,All you can eat or drink,"[greens, meat, meals, beverage, liquors]",
1,greens,Grows in a garden,,foodstuff
2,meat,Says Moooo,,foodstuff
3,meals,Is full of ingredients,,foodstuff
4,beverage,Is liquid,,foodstuff
5,liquors,Makes you dizzy,,foodstuff
6,weapons,Made for killing,"[melee, ranged, exotic]",
7,melee,To look into someone's eyes,"[pointed, blunt, edged]",weapons
8,ranged,For the introverts,,weapons
9,exotic,Is it a frying pan?,,weapons


In [7]:
mc['models'].drop_collection('items_categories')
mc['models']['items_categories'].insert_many(categories.to_dict(orient='records'))

InsertManyResult(['foodstuff', 'greens', 'meat', 'meals', 'beverage', 'liquors', 'weapons', 'melee', 'ranged', 'exotic', 'pointed', 'blunt', 'edged', 'money', 'credit', 'valuables', 'wearables', 'clothing', 'armor', 'pants', 'jackets', 'armor vests'], acknowledged=True)

In [8]:
ITEMS_PATH = '../data/items/'
items = []
for file in os.listdir(ITEMS_PATH):
  with open(ITEMS_PATH+file, 'r') as f:
    items += flatten(list(yaml.safe_load_all(f)))

In [9]:
items_sorted = {}
for v in items:
  vc = v.copy()
  del vc['cat']
  try:
    items_sorted[v['cat']].append(vc)
  except:
    items_sorted[v['cat']] = [vc]

In [10]:
for k, v in items_sorted.items():
  mc['items'].drop_collection(k)
  mc['items'][k].insert_many(v)